# Расчет проводимости трубопроводов с помощью МУК

In [1]:
import math

## Круглое сечение

In [2]:
### Без разбиения

In [3]:
class Round_section:
    def __init__(self, R, L):
        self.R = R
        self.L = L
        
    def coaxial_discs(self) -> float:
        x = self.L**2 + self.R**2 + self.R**2
        return 1/(2*self.R**2) * ( x - math.sqrt(x**2 - 4*self.R**2 * self.R**2) )
    
    def no_splitting(self) -> float:
        phi_12 = self.coaxial_discs()
        phi_21 = phi_12
        phi_11, phi_22 = 0, 0
        phi_13 = 1 - phi_11 - phi_12
        phi_23 = 1 - phi_21 - phi_22
        
        S_1 = math.pi * self.R**2
        S_3 = 2*math.pi*self.R * self.L
        phi_31 = phi_13 * S_1 / S_3
        phi_32 = phi_31
        phi_33 = 1 - phi_31 - phi_32
        
        k = phi_12 + phi_13 * phi_32 / ( 1 - phi_33 )
        return k

In [4]:
Round_section(1,2).no_splitting()

0.5857864376269049